# Introduction

In this notebook, I define a handful of functions for working with the Twitter API via Twython. These functions include `greet()`, `send_message()`, `clean_message()`, `store_messages()`, and `reply()`. I first make these functions as stand-alone entities, then combine them all into the class `Conversation`. This class is used as a chatbot that takes hard-coded inputs, and contains protocols for response when inputs are not understood.

***I have removed any API keys or user ID's that were present for app testing. If you wonder why none of these variables are visibly assigned here, that's why. Same is true for some test conversation outputs that had personal info.***

## Libraries

In [13]:
from twython import Twython
import time
import string

## Twython Object

In [15]:
# Set API version to 1.1. Version 2 doesn't have DM support yet.

t = Twython(app_key=TWITTER_APP_KEY, 
            app_secret=TWITTER_APP_KEY_SECRET, 
            oauth_token=TWITTER_ACCESS_TOKEN, 
            oauth_token_secret=TWITTER_ACCESS_TOKEN_SECRET,
            api_version='1.1')

# Functions For Chat Bot

##  Send + Receive Messages

*Note that some of these cells have had their outputs deleted for privacy.*

First we must establish the basic permissions of the app I'm working on. Most importantly, the app can send and receive tweets to and from an account that **isn't** the one the app is attached to. Here is an example of message sent:

In [1]:
t.send_direct_message(event = {"type" : "message_create",
                               "message_create" : {"target": {"recipient_id" : USER_ID},
                                                   "message_data" : {"text" : "Hey other Steven!"}}})

The app can also check it's direct messages, and return all items. Here are all the messages sent **and** received by the account the app is attached to:

In [17]:
all_messages = t.get_direct_messages()
all_messages

## Storing Received Messages

In order for "conversation" to eventually happen between a user and the chatbot, we must be able to **save messages and reply to them.** The function `store_messages()` is how that information above will be stored. This function (as well as all those that follow it) will be used in the Conversation class which will be defined later. The function `store_messages()` takes in a user ID and a set of messages. It then pulls out only the relevant tweets and saves their `time`, `user_id`, and `text`.

In [6]:
def store_messages(user_id, messages):
    
    '''
    Gets and saves all messages sent from a specific user to the bot.
    Output returns a list of dictionaries, where all keys and values are
    strings.
    ---
    user_id
        The ID of the user you want to store tweets from.
    messages
        Must be the output from Twython.get_direct_messages().
        This is what you get the tweets from.
    '''

    message_list = []
    user_id = str(user_id)
    
    for i in range(len(messages['events'])):
        
        message = messages['events'][i]
        sender_id = message['message_create']['sender_id']
        
        if sender_id == user_id:
            message_dict = {}
            message_dict['time'] = message['created_timestamp']
            message_dict['user_id'] = message['message_create']['sender_id']
            message_dict['text'] = message['message_create']['message_data']['text']
            message_list.append(message_dict)
            
    return message_list

In [3]:
got_messages = store_messages(USER_ID, all_messages)
got_messages

## Replying To Messages

Replying is a more complex process, and requires first cleaning any incoming messages and defining a protocol for sending replies. One function for each of these preliminary tasks is defined below, and the functions `clean_messages()` and `send_message()` are then incorporated into a final function `reply()`.

### Cleaning Messages

The function below removes all punctuation from an incoming message, changes the whole thing to lowercase, and returns a split version of the message that contains **only** the acceptable values for replying to the chatbot. These values are `['yes', 'no', 'help', 'stop']`.

In [8]:
def clean_message(message):
    
    '''
    Cleans out punctuation and capitals from a user message. Returns the
    string as a list of strings so that replies can be parsed easily.
    ---
    message
        Must be a string, ideally as seen in store_messages()[i]['text']
    '''
    
    allowed_replies = ['yes', 'no', 'help', 'stop']
    
    for i in string.punctuation:
        message = message.replace(i, '').lower()
    
    new_message = []
    for word in message.split():
        if word in allowed_replies:
            new_message.append(word)
            
    return new_message

In [9]:
clean_message('yeEs ,yeS No !!!hello 3456 test')

['yes', 'no']

### Sending Messages

The function below just uses the Twython function `send_direct_message()`. It is being rewritten here for **ease of use,** such that a whole JSON need not be written for each use of the function.

In [9]:
def send_message(text, user_id, t):
    
    '''
    Sends a message to a specified user.
    ---
    text
        String of what to send to user.
    user_id
        Twitter user ID. This is who gets the message.
    t
        Instance of Twython object. Contains authorization info and specifies
        API v1.1 which allows DMs to be send.
    '''
    
    t.send_direct_message(
        event = {"type" : "message_create",
                 "message_create" : {"target": {"recipient_id" : str(user_id)},
                                     "message_data" : {"text" : text}}}
    )

In [4]:
send_message(text = "This is a test of the send_message() function",
             user_id = USER_ID,
             t = t)

The above cell send this to my inbox:

![Picture](images/reply_send_message_function.PNG)

### Putting It All Together To Reply

The `reply()` function below reads the outputs from `store_messages()` and replies based on a few allowed parameters. The bot is not intended to be a fully-functioning chatbot, but rather an intermediate step to perform outreach before passing on potential clients and business partners to humans. As such, the allowed inputs form the list `['yes', 'no', 'help', 'stop']`. Each input illicits a specific reply from the bot.

> *Note: The variables* `help_count` *and* `no_contact_list`*are unused here, but will come into play in the full class.*

In [42]:
def reply(message_in, user_id, t):
    
    '''
    This function allows the bot to talk back to users. It replies to four
    possible, pre-specified inputs, and provides an alternative for when the
    user input is not in the pre-specified list.
    ---
    message_in
        Must be a string. This is what the bot replies to.
    user_id
        Twitter user ID. This is who gets the message.
    t
        Instance of Twython object. Contains authorization info and specifies
        API v1.1 which allows DMs to be send.
    '''
    
    message_in = clean_message(message_in)
    
    user_id = str(user_id)
    
    allowed_replies = ['yes', 'no', 'help', 'stop']
    
    if (len(message_in) > 1) or (len(message_in) == 0):
        send_message(text = "I'm sorry but I don't know what you want. Please reply only with YES, NO, HELP, or STOP.",
                     user_id = user_id,
                     t = t)
#         help_count += 1
        return

        
    if message_in[0] == 'yes':
        send_message(text = "We're happy to hear it! A spokesperson will be in touch shortly :)",
                     user_id = user_id,
                     t = t)
        print(f'User at ID {user_id} is interested in collaborating! Get a human on this task at one!')
        return
    
    if message_in[0] == 'no':
        send_message(text = "We're sad you won't be joining us. Have a nice day!",
                     user_id = user_id,
                     t = t)
        return

    if message_in[0] == 'help':
        send_message(text = "Reply YES to show interest in a brand deal, NO to decline, HELP to see this message, and STOP to be put on our no-contact list.",
                     user_id = user_id,
                     t = t)
        return

    if message_in[0] == 'stop':
#         no_contact = True
        return

In [5]:
reply(message_in = "help stop", user_id = USER_ID, t=t)

The cell above was a **functional test** that produced this result in my inbox:

![Picture](images/reply_im_sorry.PNG)

In [6]:
reply(message_in = "yes i'd love to work with you", user_id = USER_ID, t=t)

And this cell produced this response:

![Picture](images/reply_yes.PNG)

## Greeting Users

One last thing to do before assembling the bot is to create a simple function for greeting users. This will be run at the beginning of a conversation.

In [33]:
def greet(user_id, t, brand):
    '''
    Greets a user with the chatbot, introduces what brand the bot works for,
    and offers four options for replying to the bot.
    ---
    text
        Content of message. String.
    user_id
        User the message will go to. String.
    t
        Twython instance.
    brand
        What brand the bot is working for. String.
    '''
    text = f'Hi there! Wanna collaborate with {brand}? Please respond with YES, NO, HELP, or STOP.'
    send_message(text = text, user_id = user_id, t=t)

In [7]:
greet(user_id = USER_ID, t=t, brand='Cool Steven Brand')

This produces:

![Picture](images/reply_greet.PNG)

# `Conversation` Class

As mentioned in the introduction, the preceding functions will now be combined into the class `Conversation`. In short, this class represents the interactions between **one** Twitter user and the chatbot. Calling the functions is made easier in the class versions since many of the inputs are defined on initialization.

In [19]:
class Conversation:
    
    '''
    Class designed to be a representation of a unique conversation with one
    individual. Takes in a Twython instance, the unique user ID, and a brand
    that the bot is currently representing.
    
    Contains functions for storing, cleaning, and replying to messages.
    ---
    t
        Twython instance.
    user_id
        Twitter user ID. String or int.
    brand
        Defaults to Amazon, but can be any string.
    help_count
        Number of times Twitter user has sent "HELP" to chatbot. If count
        is >= 3, bot sets no_contact to True.
    no_contact
        Boolean. When True, bot will not reply to user.
    passed_to_human
        Whether or not the interaction has been passed on to a human employee.
        Defaults to False. When a user says "YES" they want to work with brand,
        this switches to True.
    '''
    
    def __init__(self, t, user_id, brand = 'Amazon'):
        self.t = t
        self.user_id = str(user_id)
        self.brand = brand
        self.help_count = 0
        self.messages = None
        self.no_contact = False
        self.passed_to_human = False
    
    
    
    def store_messages(self):

        '''
        Gets and saves all messages sent from Twitter user self.user_id to
        the bot. Output returns a list of dictionaries, where all keys and
        values are strings.
        '''
        
        messages = self.t.get_direct_messages()
        
        message_list = []

        for i in range(len(messages['events'])):

            message = messages['events'][i]
            sender_id = message['message_create']['sender_id']

            if sender_id == self.user_id:
                message_dict = {}
                message_dict['time'] = message['created_timestamp']
                message_dict['user_id'] = message['message_create']['sender_id']
                message_dict['text'] = message['message_create']['message_data']['text']
                message_list.append(message_dict)

        self.messages = message_list
        
        

    def clean_message(self, message):

        '''
        Cleans out punctuation and capitals from a user message. Returns the
        string as a list of strings so that replies can be parsed easily.
        ---
        message
            Must be a string, ideally as seen in store_messages()[i]['text']
        '''

        allowed_replies = ['yes', 'no', 'help', 'stop']

        for i in string.punctuation:
            message = message.replace(i, '').lower()

        new_message = []
        for word in message.split():
            if word in allowed_replies:
                new_message.append(word)

        return new_message

    

    def send_message(self, text):

        '''
        Sends a message to a specified user.
        ---
        text
            String of what to send to user.
        '''

        if self.no_contact == True:
            return 
        
        self.t.send_direct_message(
            event = {"type" : "message_create",
                     "message_create" : {"target": {"recipient_id" : self.user_id},
                                         "message_data" : {"text" : text}}}
        )
        


    def reply(self, message_in):
        
        '''
        This function allows the bot to talk back to users. It replies to four
        possible, pre-specified inputs, and provides an alternative for when the
        user input is not in the pre-specified list.
        ---
        message_in
            Must be a string. This is what the bot replies to.
        '''

        message_in = self.clean_message(message_in)
        allowed_replies = ['yes', 'no', 'help', 'stop']
        
        if self.no_contact == True:
            return
        
        if (len(message_in) > 1) or (len(message_in) == 0):
            if self.help_count >= 3:
                self.send_message("Looks like you're havin a hard' time bud. I'll leave you alone :/")
                self.no_contact = True
            else:
                self.send_message("I'm sorry but I don't know what you want. Please reply only with YES, NO, HELP, or STOP.")
            self.help_count += 1
            return

        if message_in[0] == 'yes':
            self.send_message("We're happy to hear it! A spokesperson will be in touch shortly :)")
            print(f'User at ID {self.user_id} is interested in collaborating! Get a human on this task at once!')
            self.passed_to_human = True
            return

        if message_in[0] == 'no':
            self.send_message("We're sad you won't be joining us. Have a nice day!")
            return

        if message_in[0] == 'help':
            self.send_message("Reply YES to show interest in a brand deal, NO to decline, HELP to see this message, and STOP to be put on our no-contact list.")
            self.help_count += 1
            return

        if message_in[0] == 'stop':
            self.no_contact = True
            return
    

    
    def greet(self):
        
        '''
        Greets a user with the chatbot, introduces what brand the bot works for,
        and offers four options for replying to the bot.
        ---
        All parameters pre-determined in __init__.
        '''
        
        text = f'Hi there! Wanna collaborate with {self.brand}? Please respond with YES, NO, HELP, or STOP.'
        self.send_message(text)

## Selected `Conversation` Class Examples

The new way to call these functions looks like this. Some examples were removed for privacy.

### Instantiate

In [149]:
t = Twython(app_key=TWITTER_APP_KEY, 
            app_secret=TWITTER_APP_KEY_SECRET, 
            oauth_token=TWITTER_ACCESS_TOKEN, 
            oauth_token_secret=TWITTER_ACCESS_TOKEN_SECRET,
            api_version='1.1')

In [20]:
c = Conversation(t = t, user_id = USER_ID)

### `greet()`

In [142]:
c.greet()

![Amazon Greet](images/reply_greet_amazon.PNG)

### `.messages` and `store_messages()`

In [143]:
c.messages

No output on `messages` attribute, until...

In [9]:
c.store_messages()
c.messages

### `reply()`

In [10]:
c.reply("Haha yes I'd love to")

### `.no_contact`

In [146]:
c.no_contact

False

In [147]:
c.reply("stop I hate you stupid robot :(")

In [148]:
c.no_contact

True

# Demo

This demo only makes sense as a live one with Twitter open in the other window. Feel free to ignore it.

In [8]:
t = Twython(app_key=TWITTER_APP_KEY, 
            app_secret=TWITTER_APP_KEY_SECRET, 
            oauth_token=TWITTER_ACCESS_TOKEN, 
            oauth_token_secret=TWITTER_ACCESS_TOKEN_SECRET,
            api_version='1.1')

In [11]:
c = Conversation(t = t, user_id = USER_ID)

In [10]:
c.greet()

In [17]:
c.store_messages()

In [18]:
new_message = c.messages[0]['text']

In [19]:
new_message

'yes if by collaborate you mean hehehehe ;)'

In [12]:
c.reply(new_message)